# IN PROGRESS

In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [4]:
###Set notebook preferences##
#Read in some libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import Image

#Set notebook preferences - pandas
pd.set_option("display.max_columns", 101)

#Set path to modeling folder
model_viz = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Spain Hourly Energy Demand and Weather\Visualizations\Modeling'

#Set path to data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Spain Hourly Energy Demand and Weather\Data\02_Cleaned_Data'

# 1. Introduction

Airbnb is easily one of the most successful start-ups in recent memory. Airbnbn's business model has been a disruptive force in the short-term rental market and was valued at \\$31 billion prior to the Corona Virus epidemic ([source](https://www.reuters.com/article/us-airbnb-debt/airbnbs-new-1-billion-investment-comes-at-lower-valuation-sources-idUSKBN21P3IM#:~:text=(Reuters)%20%2D%20Airbnb%20Inc's%20new,the%20matter%20said%20on%20Tuesday.&text=Airbnb%20was%20valued%20at%20%2431,recent%20private%20fund%2Draising%20round.)).
Howeevern in a recent artivle in April(just after massive lockdowns swept the world), the industry titans valuation has plummeted to nearly half ([source](https://skift.com/2020/04/08/airbnbs-valuation-almost-halved-to-18-billion/)).

In this analysis, I will be exploring the Airbnb rental market in San Francisco from 7/1/2018 through 6/7/2020. I intend to develop a better understanding about the impact COVID-19 is having on the SF market.

## 1.1 Intent

## 1.2 Objectives

- Conduct an analysis of the impact COVID-19 has had on daily gross rental income in the San Francisco area since the lockdown began on 3/19/2020. Forecast 2020 Q2, Q3, and Q4 losses if lockdown conditions remain the same for the rest of the year.
- Create a per night rental price predictor for Airbnb properties in San Francisco. Extract out the most pertinent features that predict prices best. 

## 1.3 Impact

I hope to use the information gained from the analysis to provide the following:
- Proojectug losses for the year will help company leadership manage expecations for profist and table/priotitze certain projects over others.
- Provide first-time hosts a guide of features/services to consider including in their Airbnb listing to increase the liklihood of interests from tourists as well as justify a higher rental price. Developing a good price predictor will also provide renters a range of acceptavble prices to list their property for(as compared to similar properties in SF)

# 2. Table of Contents

1. **Introduction**<br>
   1.1 Intent<br>
   1.2 Objectives<br>
   1.3 Impact<br>
2. **Table of Contents**<br>
3. **Data Overview**<br>
    3.1 About the Data<br>
    3.2 Preview Data<br>
    3.3 Data Overview - Metrics<br>
4. **Impact of COVID 19 on Gross Rental Revenue**<br>
    4.1 Methodology<br>
    4.2 Results<br>
    4.3 Decision Recommendations<br>
5. **Airbnb Rental Prices in San Francisco**<br>
    5.1 Methodology<br>
    5.2 Original TSO Forecast<br>
    5.3 Mulitvariate XGBRegressor Forecast<br>
    5.4 Decision Recommendations<br>
6. **Conclusion**<br>
    6.1 Takeaways<br>
    6.2 Future Research<br>

# 3. Data Overview

## 3.1 About the Data

Airbnb Listings data was acquired from Inside Airbnb(no affiliation with Airbnb). Inside Airbnb is an independent, non-commercial set of tools and data that allows you to explore how Airbnb is really being used in cities around the world.

By analyzing publicly available information about a city's Airbnb's listings, Inside Airbnb provides filters and key metrics so you can see how Airbnb is being used to compete with the residential housing market.

## 3.2 Preview Data

In [11]:
#Set path to processed data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Airbnb - San Francisco Listings Analysis\Data\02_Cleaned'

#Read CSV
df= pd.read_csv(path + '/2020_0715_Listings_Cleaned.csv')

#Preview
print('Cleaned Data Shape:',df.shape)
display(df.head())

Cleaned Data Shape: (126258, 48)


,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,cancellation_policy,city,cleaning_fee,extra_people,guests_included,host_has_profile_pic,host_id,host_identity_verified,host_is_superhost,host_name,host_response_rate,host_response_time,host_verifications,id,instant_bookable,is_location_exact,last_review,latitude,longitude,maximum_nights,minimum_nights,monthly_price,neighbourhood_cleansed,number_of_reviews,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_rating,reviews_per_month,room_type,security_deposit,weekly_price,zipcode
0,3,"TV, Cable TV ,Internet,Wifi,Kitchen, Pets liv...",0,77,0,1,1.0,Real Bed,1.0,2.0,1,0.0,0.0,moderate,San Francisco,100.0,25.0,2,1.0,1169,1.0,1.0,Holly,100.0,within an hour,"email , phone , facebook , reviews , kba",958,1,1,2019-03-16,37.76931,-122.43386,30,1,4200.0,Western Addition,183,170.0,Apartment,0,0,1,97.0,1.55,Entire home/apt,100.0,1120.0,94117.0
1,2,"TV,Internet,Wifi,Kitchen, Free street parking...",30,365,60,90,4.0,Real Bed,1.0,1.0,9,9.0,0.0,strict 14 with grace period,San Francisco,50.0,12.0,1,1.0,21994,1.0,0.0,Aaron,100.0,within a few hours,"email , phone , reviews , jumio , govern...",8142,0,1,2018-09-12,37.76487,-122.45183,90,32,1685.0,Haight Ashbury,8,65.0,Apartment,0,0,1,93.0,0.14,Private room,200.0,490.0,94117.0
2,5,"TV,Internet,Wifi,Kitchen,Heating, Family/kid ...",30,90,60,90,1.5,Real Bed,2.0,2.0,2,0.0,0.0,strict 14 with grace period,San Francisco,225.0,150.0,2,1.0,24215,1.0,0.0,Rosy,100.0,within an hour,"email , phone , reviews , kba",8339,0,1,2018-08-11,37.77525,-122.43637,1125,7,NaN,Western Addition,27,785.0,House,1,1,1,97.0,0.23,Entire home/apt,0.0,NaN,94117.0
3,6,"TV, Cable TV ,Internet,Wifi,Kitchen, Free par...",0,20,5,9,1.0,Real Bed,2.0,3.0,1,0.0,0.0,moderate,San Francisco,125.0,0.0,1,1.0,25601,0.0,0.0,Sandy,90.0,within a day,"email , phone , facebook , reviews",8567,0,1,2019-03-30,37.78471,-122.44555,365,2,NaN,Western Addition,32,255.0,Apartment,0,0,1,90.0,0.27,Entire home/apt,0.0,NaN,94115.0
4,3,"TV, Cable TV ,Internet,Wifi, Paid parking off...",4,131,19,42,1.0,Real Bed,1.0,2.0,2,2.0,0.0,strict 14 with grace period,San Francisco,50.0,60.0,2,1.0,7149,0.0,1.0,Ivan & Wendy,100.0,within an hour,"email , phone , reviews , kba , work ema...",8739,1,1,2019-03-23,37.75919,-122.42237,14,1,9999.0,Mission,656,139.0,Condominium,0,0,1,98.0,5.57,Private room,0.0,NaN,94110.0


## 3.3 Data Overview - Metrics

**Data Description**

In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
accommodates,126258.0,3.217230e+00,1.947620e+00,1.000000,2.000000e+00,2.000000e+00,4.000000e+00,1.600000e+01
availability_30,126258.0,9.534984e+00,1.049743e+01,0.000000,0.000000e+00,5.000000e+00,1.700000e+01,3.000000e+01
availability_365,126258.0,1.430715e+02,1.260210e+02,0.000000,2.500000e+01,1.070000e+02,2.630000e+02,3.650000e+02
availability_60,126258.0,2.290179e+01,2.104954e+01,0.000000,1.000000e+00,1.800000e+01,4.100000e+01,6.000000e+01
availability_90,126258.0,3.827428e+01,3.168983e+01,0.000000,5.000000e+00,3.500000e+01,6.600000e+01,9.000000e+01
bathrooms,126258.0,1.341998e+00,8.540870e-01,0.000000,1.000000e+00,1.000000e+00,1.500000e+00,1.400000e+01
bedrooms,126258.0,1.354290e+00,9.212903e-01,0.000000,1.000000e+00,1.000000e+00,2.000000e+00,1.400000e+01
beds,126258.0,1.768339e+00,1.206100e+00,0.000000,1.000000e+00,1.000000e+00,2.000000e+00,1.600000e+01
calculated_host_listings_count,126258.0,1.164335e+01,3.751799e+01,1.000000,1.000000e+00,2.000000e+00,4.000000e+00,3.010000e+02
calculated_host_listings_count_private_rooms,105994.0,2.726702e+00,7.322385e+00,0.000000,0.000000e+00,0.000000e+00,2.000000e+00,8.700000e+01


# 4. Impact of COVID 19 on Gross Rental Revenue

## 4.1 Methodology

## 4.2 Results

In [ ]:
#Read in Silhouette_Scores
Image(filename= model_viz + '/Consumer_Energy_Profiles.png', height=700, width=700)

## 4.3 Decision Recommendations

# 5. Airbnb Rental Prices in San Francisco

## 5.1 Methodology

## 5.2 Base Model Decision Tree

## 5.3 Tuned XGBRegressor Price Predictor

## 5.4 Decision Recommendations

# 6. Conclusion

## 6.1 Takeaways

## 6.2 Future Research